# Pi-HEaaN

In [2]:
%pip install pi-heaan
%pip install numpy
%pip install pandas

import piheaan as heaan
from piheaan.math import sort
from piheaan.math import approx # for piheaan math function
import math
import numpy as np
import pandas as pd
import os

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# set parameter
params = heaan.ParameterPreset.FGb
context = heaan.make_context(params) # context has paramter information
heaan.make_bootstrappable(context) # make parameter bootstrapable

# create and save keys
key_file_path = "./keys"
sk = heaan.SecretKey(context) # create secret key
os.makedirs(key_file_path, mode=0o775, exist_ok=True)
sk.save(key_file_path+"/secretkey.bin") # save secret key

key_generator = heaan.KeyGenerator(context, sk) # create public key
key_generator.gen_common_keys()
key_generator.save(key_file_path+"/") # save public key

In [4]:
# load secret key and public key
# When a key is created, it can be used again to save a new key without creating a new one
key_file_path = "./keys"

sk = heaan.SecretKey(context,key_file_path+"/secretkey.bin") # load secret key
pk = heaan.KeyPack(context, key_file_path+"/") # load public key
pk.load_enc_key()
pk.load_mult_key()

eval = heaan.HomEvaluator(context,pk) # to load piheaan basic function
dec = heaan.Decryptor(context) # for decrypt
enc = heaan.Encryptor(context) # for encrypt

In [5]:
# log_slots is used for the number of slots per ciphertext
# It depends on the parameter used (ParameterPreset)
# The number '15' is the value for maximum number of slots,
# but you can also use a smaller number (ex. 2, 3, 5, 7 ...)
# The actual number of slots in the ciphertext is calculated as below.
log_slots = 15 
num_slots = 2**log_slots

- The computations in pi-heaan are based on operations between slots.
- Also, the result of an operation between ciphertexts is placed into a new ciphertext (or overwritten).

# 01 heaan.HomEvaluator

### add
- add : addition of two ciphertexts
    - add(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext, arg2: piheaan.Ciphertext)
    - add(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Message, arg2: piheaan.Ciphertext)
    - add(self: piheaan.HomEvaluator, arg0: piheaan.Message, arg1: piheaan.Message, arg2: piheaan.Message)
    - arg2 <- arg0 + arg1    

### sub
- sub : subtraction of two ciphertexts
    - same paramters as 'add'

In [6]:
log_slots = 3
num_slots = 2**log_slots

data1 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
message_1 = heaan.Message(log_slots)
for i in range(num_slots):
    message_1[i] = data1[i]

data2 = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.1]
message_2 = heaan.Message(log_slots)
for i in range(num_slots):
    message_2[i] = data2[i]


print("message_1 : ", message_1)
print()
print("message_2 : ", message_2)

message_1 :  [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j) ]

message_2 :  [ (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j) ]


In [11]:
# (ciphertext + ciphertext)
ciphertext_1 = heaan.Ciphertext(context)
ciphertext_2 = heaan.Ciphertext(context)
result_add = heaan.Ciphertext(context)

enc.encrypt(message_1, pk, ciphertext_1)
enc.encrypt(message_2, pk, ciphertext_2)
print(ciphertext_1)
print(ciphertext_2)

eval.add(ciphertext_1, ciphertext_2, result_add)
print(result_add)

result_add_message = heaan.Message(log_slots)
dec.decrypt(result_add, sk, result_add_message)

print("(ciphertext + ciphertext) : ", result_add_message)


(level: 12, log(num slots): 3, data: [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j) ])
(level: 12, log(num slots): 3, data: [ (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j) ])
(level: 12, log(num slots): 3, data: [ (0.300000+0.000000j), (0.500000+0.000000j), (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (1.500000+0.000000j), (0.900000+0.000000j) ])
(ciphertext + ciphertext) :  [ (0.300000+0.000000j), (0.500000+0.000000j), (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (1.500000+0.000000j), (0.900000+0.000000j) ]


In [7]:
# (ciphertext + message) 1
eval.add(ciphertext_1, message_2, result_add)

result_add_message = heaan.Message(log_slots)
dec.decrypt(result_add, sk, result_add_message)

print("(ciphertext + message) : ", result_add_message)

(ciphertext + message) :  [ (0.300000+0.000000j), (0.500000+0.000000j), (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (1.500000+0.000000j), (0.900000+0.000000j) ]


In [8]:
# (ciphertext + message(int or float or ...)) 2
eval.add(ciphertext_1, 2, result_add)

result_add_message = heaan.Message(log_slots)
dec.decrypt(result_add, sk, result_add_message)

print("(ciphertext + message) : ", result_add_message)

(ciphertext + message) :  [ (2.100000+0.000000j), (2.200000+0.000000j), (2.300000+0.000000j), (2.400000+0.000000j), (2.500000+0.000000j), (2.600000+0.000000j), (2.700000+0.000000j), (2.800000+0.000000j) ]


In [9]:
# (message + message)
result_add_message = heaan.Message(log_slots)
eval.add(message_1, message_2, result_add_message)

print("(message + message) : ", result_add_message)

(message + message) :  [ (0.300000+0.000000j), (0.500000+0.000000j), (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (1.500000+0.000000j), (0.900000+0.000000j) ]


### mult
- multiplication of two ciphertexts
    - mult(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext, arg2: piheaan.Ciphertext)
    - mult(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Message, arg2: piheaan.Ciphertext)
    - mult(self: piheaan.HomEvaluator, arg0: piheaan.Message, arg1: piheaan.Message, arg2: piheaan.Message)

In [10]:
# (ciphertext * ciphertext)
result_mult = heaan.Ciphertext(context)
eval.mult(ciphertext_1, ciphertext_2, result_mult)

result_mult_message = heaan.Message(log_slots)
dec.decrypt(result_mult, sk, result_mult_message)

print("(ciphertext * ciphertext) : ", result_mult_message)

(ciphertext * ciphertext) :  [ (0.020000+0.000000j), (0.060000+0.000000j), (0.120000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.420000+0.000000j), (0.560000+0.000000j), (0.080000+0.000000j) ]


In [11]:
# (ciphertext * message) 1
eval.mult(ciphertext_1, message_2, result_mult)

result_mult_message = heaan.Message(log_slots)
dec.decrypt(result_mult, sk, result_mult_message)

print("(ciphertext * message) : ", result_mult_message)

(ciphertext * message) :  [ (0.020000+0.000000j), (0.060000+0.000000j), (0.120000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.420000+0.000000j), (0.560000+0.000000j), (0.080000+0.000000j) ]


In [12]:
# (ciphertext * message) 2
eval.mult(ciphertext_1, 2, result_mult)

result_mult_message = heaan.Message(log_slots)
dec.decrypt(result_mult, sk, result_mult_message)

print("(ciphertext * ciphertext) : ", result_mult_message)

(ciphertext * ciphertext) :  [ (0.200000+0.000000j), (0.400000+0.000000j), (0.600000+0.000000j), (0.800000+0.000000j), (1.000000+0.000000j), (1.200000+0.000000j), (1.400000+0.000000j), (1.600000+0.000000j) ]


In [13]:
# (message * message)
result_mult_message = heaan.Message(log_slots)
eval.mult(message_1, message_2, result_mult_message)

print("(message * message) : ", result_mult_message)

(message * message) :  [ (0.020000+0.000000j), (0.060000+0.000000j), (0.120000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.420000+0.000000j), (0.560000+0.000000j), (0.080000+0.000000j) ]


### negate
- change the sign (ex. -2 -> +2)
- negate(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext)

In [14]:
result_negate = heaan.Ciphertext(context)

# plus sign -> minus sign
eval.negate(ciphertext_1, result_negate)

result_negate_message = heaan.Message(log_slots)
dec.decrypt(result_negate, sk, result_negate_message)
print("-(ciphertext_1) : ", result_negate_message)
print()

# minus sign -> plus sign
eval.negate(result_negate, result_negate)

result_negate_message = heaan.Message(log_slots)
dec.decrypt(result_negate, sk, result_negate_message)
print("-(-ciphertext_1) : ", result_negate_message)

-(ciphertext_1) :  [ (-0.100000+0.000000j), (-0.200000+0.000000j), (-0.300000+0.000000j), (-0.400000+0.000000j), (-0.500000+0.000000j), (-0.600000+0.000000j), (-0.700000+0.000000j), (-0.800000+0.000000j) ]

-(-ciphertext_1) :  [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j) ]


### square
- square of ciphertext (ex. x -> x^2)
- square(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: piheaan.Ciphertext)

In [15]:
result_square = heaan.Ciphertext(context)

eval.square(ciphertext_1, result_square)

result_square_message = heaan.Message(log_slots)
dec.decrypt(result_square, sk, result_square_message)
print("(ciphertext_1)**2 : ", result_square) 
# is the same result as "eval.mult(ciphertext_1, ciphertext_1, result_square)"

(ciphertext_1)**2 :  (level: 11, log(num slots): 3, data: [ (0.010000+0.000000j), (0.040000+0.000000j), (0.090000+0.000000j), (0.160000+0.000000j), (0.250000+0.000000j), (0.360000+0.000000j), (0.490000+0.000000j), (0.640000+0.000000j) ])


### left_rotate / right_rotate
- the operation of rotating data in ciphertext
- left(right)_rotate(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, arg1: int, arg2: piheaan.Ciphertext)

In [15]:
result_left_rot = heaan.Ciphertext(context)
result_right_rot = heaan.Ciphertext(context)

# left_rotate
eval.left_rotate(ciphertext_1, 3, result_left_rot)
print(ciphertext_1)
print(result_left_rot)

result_left_rot_message = heaan.Message(log_slots)
dec.decrypt(result_left_rot, sk, result_left_rot_message)
print("left_rotate : ", result_left_rot_message)
print()

# right_rotate
eval.right_rotate(ciphertext_1, 3, result_right_rot)

result_right_rot_message = heaan.Message(log_slots)
dec.decrypt(result_right_rot, sk, result_right_rot_message)
print("right_rotate : ", result_right_rot_message)


(level: 12, log(num slots): 3, data: [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j) ])
(level: 12, log(num slots): 3, data: [ (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j) ])
left_rotate :  [ (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j) ]

right_rotate :  [ (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j) ]


### rot_sum
- rotate and add several distinct ciphertexts
- rot_sum(self: piheaan.HomEvaluator, 
            arg0: List[piheaan.Ciphertext], 
            arg1: List[int], 
            arg2: piheaan.Ciphertext)

- Type of 0th and 1st parameters are 'List'
- The order and position of arg0 and arg1 must be matched
    - (1) eval.rot_sum([ctxt1, ctxt2], [2], ctxt3) => is only the result left rotate 2 slots for ctxt1
    - (2) eval.rot_sum([ctxt1], [2, 3], ctxt3) => error!

In [17]:
result_rot_sum = heaan.Ciphertext(context)

eval.rot_sum([ciphertext_1, ciphertext_2], [2, 2], result_rot_sum) 

result_rot_sum_message = heaan.Message(log_slots)
dec.decrypt(result_rot_sum, sk, result_rot_sum_message)
print("rot_sum : ", result_rot_sum_message)

rot_sum :  [ (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (1.500000+0.000000j), (0.900000+0.000000j), (0.300000+0.000000j), (0.500000+0.000000j) ]


In [18]:
# redeclaration
ciphertext_1 = heaan.Ciphertext(context)
ciphertext_2 = heaan.Ciphertext(context)
enc.encrypt(message_1, pk, ciphertext_1)
enc.encrypt(message_2, pk, ciphertext_2)

print("ciphertext_1 : ", message_1)
print("ciphertext_2 : ", message_2)
print()

# "rot_sum" performs the following process at once
res1 = heaan.Ciphertext(context)
res2 = heaan.Ciphertext(context)
eval.left_rotate(ciphertext_1, 2, res1)
eval.left_rotate(ciphertext_2, 2, res2)
eval.add(res1, res2, result_rot_sum)

message_ = heaan.Message(log_slots)
dec.decrypt(result_rot_sum, sk, message_)
print("result : ", message_)

ciphertext_1 :  [ (0.100000+0.000000j), (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j) ]
ciphertext_2 :  [ (0.200000+0.000000j), (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j) ]

result :  [ (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (1.500000+0.000000j), (0.900000+0.000000j), (0.300000+0.000000j), (0.500000+0.000000j) ]


In [19]:
# (1)
result_rot_sum = heaan.Ciphertext(context)

eval.rot_sum([ciphertext_1, ciphertext_2], [2], result_rot_sum) 

result_rot_sum_message = heaan.Message(log_slots)
dec.decrypt(result_rot_sum, sk, result_rot_sum_message)
print("(1) rot_sum : ", result_rot_sum_message)

(1) rot_sum :  [ (0.300000+0.000000j), (0.400000+0.000000j), (0.500000+0.000000j), (0.600000+0.000000j), (0.700000+0.000000j), (0.800000+0.000000j), (0.100000+0.000000j), (0.200000+0.000000j) ]


### left_rotate_reduce / right_rotate_reduce
- You can use it in many rotating ways
- left_rotate_reduce(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, 
	       arg1: int, arg2: int, arg3: piheaan.Ciphertext)
- left_rotate_reduce works the same way as right_rotate_reduce with the different rotate direction

In [20]:
# redeclaration
ciphertext_1 = heaan.Ciphertext(context)
ciphertext_2 = heaan.Ciphertext(context)
enc.encrypt(message_1, pk, ciphertext_1)
enc.encrypt(message_2, pk, ciphertext_2)

In [21]:
# (1)
msg_result_rigt_rot_reduce1=heaan.Message(log_slots)
result_right_rot_reduce1 = heaan.Ciphertext(context)
enc.encrypt(msg_result_rigt_rot_reduce1, pk,result_right_rot_reduce1 )

eval.right_rotate_reduce(ciphertext_1, 1, 5, result_right_rot_reduce1)

result_right_rot_reduce_message = heaan.Message(log_slots)
dec.decrypt(result_right_rot_reduce1, sk, result_right_rot_reduce_message)
print("(1) right_rotate_reduce : ", result_right_rot_reduce_message)
print()

# This is the same result..

msg_result_rigt_rot_reduce1=heaan.Message(log_slots)
result_right_rot_reduce1 = heaan.Ciphertext(context)
enc.encrypt(msg_result_rigt_rot_reduce1, pk,result_right_rot_reduce1 )

for i in range(5):
    if i>0:
        eval.right_rotate(ciphertext_1, 1, ciphertext_1)
    eval.add(ciphertext_1, result_right_rot_reduce1, result_right_rot_reduce1)

result_right_rot_reduce_message = heaan.Message(log_slots)
dec.decrypt(result_right_rot_reduce1, sk, result_right_rot_reduce_message)
print("(1) same result : ", result_right_rot_reduce_message)

(1) right_rotate_reduce :  [ (2.700000+0.000000j), (2.400000+0.000000j), (2.100000+0.000000j), (1.800000+0.000000j), (1.500000+0.000000j), (2.000000+0.000000j), (2.500000+0.000000j), (3.000000+0.000000j) ]

(1) same result :  [ (2.700000+0.000000j), (2.400000+0.000000j), (2.100000+0.000000j), (1.800000+0.000000j), (1.500000+0.000000j), (2.000000+0.000000j), (2.500000+0.000000j), (3.000000+0.000000j) ]


In [22]:
# redeclaration
ciphertext_1 = heaan.Ciphertext(context)
ciphertext_2 = heaan.Ciphertext(context)
enc.encrypt(message_1, pk, ciphertext_1)
enc.encrypt(message_2, pk, ciphertext_2)

In [23]:
# (2)
msg_result_rigt_rot_reduce2=heaan.Message(log_slots)
result_right_rot_reduce2 = heaan.Ciphertext(context)
enc.encrypt(msg_result_rigt_rot_reduce2, pk,result_right_rot_reduce2 )


eval.right_rotate_reduce(ciphertext_2, 5, 2, result_right_rot_reduce2)

result_right_rot_reduce_message = heaan.Message(log_slots)
dec.decrypt(result_right_rot_reduce2, sk, result_right_rot_reduce_message)
print("(2) right_rotate_reduce : ", result_right_rot_reduce_message)
print()


# This is the same result..
msg_result_rigt_rot_reduce2=heaan.Message(log_slots)
result_right_rot_reduce2 = heaan.Ciphertext(context)
enc.encrypt(msg_result_rigt_rot_reduce2, pk,result_right_rot_reduce2 )
for i in range(2):
    if i>0:
        eval.right_rotate(ciphertext_2, 5, ciphertext_2)
    eval.add(ciphertext_2, result_right_rot_reduce2, result_right_rot_reduce2)

result_right_rot_reduce_message = heaan.Message(log_slots)
dec.decrypt(result_right_rot_reduce2, sk, result_right_rot_reduce_message)
print("(2) same result : ", result_right_rot_reduce_message)

(2) right_rotate_reduce :  [ (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (0.500000+0.000000j) ]

(2) same result :  [ (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (1.300000+0.000000j), (0.700000+0.000000j), (0.900000+0.000000j), (1.100000+0.000000j), (0.500000+0.000000j) ]


In [24]:
# (3)
# if you want to calculate the sum of ctxt1, you can use this!
data_masking_1 = [1] + [0]*(num_slots-1)
masking_1 = heaan.Message(log_slots)
for i in range(num_slots):
    masking_1[i] = data_masking_1[i]

res3 = heaan.Ciphertext(context)

eval.left_rotate_reduce(ciphertext_1, 1, num_slots, res3)
eval.mult(res3, masking_1, res3)

res3_message = heaan.Message(log_slots)
dec.decrypt(res3, sk, res3_message)
print("(3) result : ", res3_message)

(3) result :  [ (3.600000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]


### bootstrap / min_level_for_bootstrap
- the level of ciphertext is a value that indicates how many times multiplication can be performed using this ciphertext
- if the level of ciphertext becomes min_level_for_bootstrap, you have to do "bootstrap"
- if the level of ciphertext exceeds 'min_level_for_bootstrap', you cannot perform bootstrap.
- Bootstrap must be done before the level reaches 3 or below
- Then the level is restored
- So you can continue the operation continuously

- bootstrap(self: piheaan.HomEvaluator, arg0: piheaan.Ciphertext, 
		arg1: piheaan.Ciphertext)

In [25]:
# Lets down the level of ciphertext
result_mult = heaan.Ciphertext(context)

eval.mult(ciphertext_1, ciphertext_1, result_mult)
print("1) check ctxt3 level : ", result_mult.level) # if you print 'ciphertext.level', you can check the level of ciphertext
for i in range(result_mult.level - eval.min_level_for_bootstrap):
    eval.mult(result_mult, result_mult, result_mult)
print("2) after 10 times mult.. ctxt3 level : ", result_mult.level) # 23 -> 3
# Now you cannot do multiplication!
# So now..
eval.bootstrap(result_mult, result_mult)
print("3) after bootstrap! ctxt3 level : ", result_mult.level) # now you can do multiplication 12 times

1) check ctxt3 level :  11
2) after 10 times mult.. ctxt3 level :  3
3) after bootstrap! ctxt3 level :  12


In [26]:
# you can check minmum level for bootstrap
eval.min_level_for_bootstrap

3

# 02 heaan.math.sort

### sort
- input range : -0.5 ~ 0.5
- 4th parameter(boolean) : False = descending, True = ascending
- 5th parameter(boolean) is False(default)
- sort(arg0: piheaan.HomEvaluator, 
        arg1: input : piheaan.Ciphertext, 
        arg2: result : piheaan.Ciphertext, 
        arg3: n : int, 
        arg4: ascent : bool, 
        arg5: only_last_stage : bool)

In [27]:
log_slots = 3
num_slots = 2 ** log_slots

data3 = np.random.uniform(size=num_slots)-0.5
message_3 = heaan.Message(log_slots)
for i in range(num_slots):
    message_3[i]=data3[i]
print(message_3)

ciphertext_3 = heaan.Ciphertext(context)
enc.encrypt(message_3, pk, ciphertext_3)

[ (-0.213592+0.000000j), (0.248106+0.000000j), (0.446777+0.000000j), (-0.153213+0.000000j), (0.452811+0.000000j), (0.258801+0.000000j), (-0.011590+0.000000j), (0.429461+0.000000j) ]


In [28]:
# Input range : -0.5 ~ 0.5

ciphertext_out_sort = heaan.Ciphertext(context)
sort.sort(eval, ciphertext_3, ciphertext_out_sort, num_slots, True) # 4th boolean: 0: descending, 1 : ascending order

message_out_sort = heaan.Message(log_slots)

dec.decrypt(ciphertext_out_sort, sk, message_out_sort)

print("sort : ", message_out_sort)

index pair in unitSort :0, 1
index pair in unitSort :1, 2
index pair in unitSort :1, 1
index pair in unitSort :2, 4
index pair in unitSort :2, 2
index pair in unitSort :2, 1
sort :  [ (-0.213592+0.000000j), (-0.153213+0.000000j), (-0.011590+0.000000j), (0.248106+0.000000j), (0.258801+0.000000j), (0.429461+0.000000j), (0.446777+0.000000j), (0.452811+0.000000j) ]


# 03 heaan.math.approx

### compare
- compare the slots of different ciphertexts and identify which value is greater or smaller
- input range : 2^-18 < |x-y| < 1
- compare(arg0: piheaan.HomEvaluator, 
            arg1: piheaan.Ciphertext, 
            arg2: piheaan.Ciphertext, 
            arg3: piheaan.Ciphertext)

- if the value in slot of arg1 > arg2, then return 1
- if the value in slot of arg1 < arg2, then return 0
- if the value in slot of arg1 == arg2, then return 0.5

In [29]:
data3 = np.random.uniform(size=num_slots)
data4 = np.random.uniform(size=num_slots)
message_3 = heaan.Message(log_slots)
message_4 = heaan.Message(log_slots)
for i in range(num_slots):
    message_3[i]=data3[i]
    message_4[i]=data4[i]

print("message_3 : ", message_3)
print()
print("message_4 : ", message_4)

message_3 :  [ (0.316636+0.000000j), (0.522070+0.000000j), (0.218255+0.000000j), (0.871919+0.000000j), (0.167986+0.000000j), (0.522926+0.000000j), (0.438431+0.000000j), (0.846082+0.000000j) ]

message_4 :  [ (0.519363+0.000000j), (0.179869+0.000000j), (0.573564+0.000000j), (0.809370+0.000000j), (0.418579+0.000000j), (0.358110+0.000000j), (0.207771+0.000000j), (0.711561+0.000000j) ]


In [30]:
ciphertext_3 = heaan.Ciphertext(context)
ciphertext_4 = heaan.Ciphertext(context)
result_compare = heaan.Ciphertext(context)

enc.encrypt(message_3, pk, ciphertext_3)
enc.encrypt(message_4, pk, ciphertext_4)

In [31]:
approx.compare(eval, ciphertext_3, ciphertext_4, result_compare)

result_comp_message = heaan.Message(log_slots)
dec.decrypt(result_compare, sk, result_comp_message)
print("result_comp_message : ", result_comp_message)

result_comp_message :  [ (0.000000+0.000000j), (1.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j) ]


### discrete_equal
- compare two ciphertexts and identify whether they have the smae value
- input range : |x| ≤ 54 (x : int)
- compare two ciphertexts and identify whether they have the same value
- discrete_equal(eval: piheaan.HomEvaluator, 
                    op1: piheaan.Ciphertext, 
                    op2: piheaan.Ciphertext, 
                    res: piheaan.Ciphertext)


### discrete_equal_zero
- compare a ciphertest with 0 and identify whether the value of the ciphertext is equal to 0
- input range : |x| ≤ 54 (x : int)
- discrete_equal_zero(arg0: piheaan.HomEvaluator, 
                      arg1: piheaan.Ciphertext, 
		              arg2: piheaan.Ciphertext)

In [32]:
data5 = [i for i in range(num_slots)]
data6 = [0, 1, 2, 3, 0, 0, 2, 0]
message_5 = heaan.Message(log_slots)
message_6 = heaan.Message(log_slots)
for i in range(num_slots):
    message_5[i]=data5[i]
    message_6[i]=data6[i]

print("message_5 : ", message_5)
print()
print("message_6 : ", message_6)

message_5 :  [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (4.000000+0.000000j), (5.000000+0.000000j), (6.000000+0.000000j), (7.000000+0.000000j) ]

message_6 :  [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (2.000000+0.000000j), (0.000000+0.000000j) ]


In [33]:
ciphertext_5 = heaan.Ciphertext(context)
ciphertext_6 = heaan.Ciphertext(context)

enc.encrypt(message_5, pk, ciphertext_5)
enc.encrypt(message_6, pk, ciphertext_6)

# ciphertext5_left_rotate = heaan.Ciphertext(context)  
# eval.left_rotate(ciphertext_5, 5, ciphertext5_left_rotate)  # 5번 암호문을 좌측으로 5칸 로테이션

result_discrete_equal = heaan.Ciphertext(context)
result_discrete_equal_zero = heaan.Ciphertext(context)
approx.discrete_equal(eval, ciphertext_5, ciphertext_6, result_discrete_equal)
approx.discrete_equal_zero(eval, ciphertext_6, result_discrete_equal_zero)
# approx.discrete_equal(eval, ciphertext_5, ciphertext5_left_rotate, result_discrete_equal_rotate)

result_discrete_equal_message = heaan.Message(log_slots)
dec.decrypt(result_discrete_equal, sk, result_discrete_equal_message) # 5번과 6번 메시지 비교

result_discrete_equal_zero_message = heaan.Message(log_slots)
dec.decrypt(result_discrete_equal_zero, sk, result_discrete_equal_zero_message) # 5번과 로테이션 비교

print('message_5 :',message_5)
print()
print('message_6 :', message_6)
print()
print('discrete_equal result : ', result_discrete_equal_message)
print()
print('discrete_equal_zero result : ', result_discrete_equal_zero_message)

message_5 : [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (4.000000+0.000000j), (5.000000+0.000000j), (6.000000+0.000000j), (7.000000+0.000000j) ]

message_6 : [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (2.000000+0.000000j), (0.000000+0.000000j) ]

discrete_equal result :  [ (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j) ]

discrete_equal_zero result :  [ (1.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j) ]


### inverse
- take the inverse of the value of the ciphertext
- input range : 1 ≤ x ≤ 2^22 or 2^-10 ≤ x ≤ 1
- inverse(arg0: piheaan.HomEvaluator, 
          arg1: piheaan.Ciphertext, 
		  arg2: piheaan.Ciphertext)

          
- inverse(eval: piheaan.HomEvaluator, 
            op: piheaan.Ciphertext, 
            res: piheaan.Ciphertext, 
            init: Optional[float] = None, 
            num_iter: Optional[int] = None, 
            greater_than_one: bool = True)
            
- defaul values are init=pow(2, -18), num_iter=23 if greater_than_one==true,
- otherwise init=pow(2, -10), num_iter=15

In [34]:
data = [i for i in range(1, num_slots+1)]
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]

In [35]:
ciphertext = heaan.Ciphertext(context)
result_inv = heaan.Ciphertext(context)

enc.encrypt(message, pk, ciphertext)
approx.inverse(eval, ciphertext, result_inv) 

decryptor = heaan.Decryptor(context)
result_inv_message = heaan.Message(log_slots)

decryptor.decrypt(result_inv, sk, result_inv_message)

print('inverse :', result_inv_message) # 

inverse : [ (1.000000+0.000000j), (0.500000+0.000000j), (0.333333+0.000000j), (0.250000+0.000000j), (0.200000+0.000000j), (0.166667+0.000000j), (0.142857+0.000000j), (0.125000+0.000000j) ]


### log
- input range : 1 ≤ x ≤ 2^22
- approx.log : 3th parameter is the number 'base' (ex. 2, 10, e, ...)
- log(arg0: piheaan.HomEvaluator, 
        arg1: input : piheaan.Ciphertext, 
        arg2: result : piheaan.Ciphertext, 
        arg3: float)


In [36]:
log_slots = 3
num_slots = 2 ** log_slots

data = [i for i in range(num_slots)]
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]

ciphertext = heaan.Ciphertext(context)
enc.encrypt(message, pk, ciphertext)

In [37]:
ciphertext_log_2 = heaan.Ciphertext(context)
ciphertext_log_10 = heaan.Ciphertext(context)
ciphertext_log_e = heaan.Ciphertext(context)
ciphertext_log_ = heaan.Ciphertext(context)

approx.log(eval, ciphertext, ciphertext_log_, 2)
approx.log_2(eval, ciphertext, ciphertext_log_2)  #  1 ~ 2^18
approx.log_10(eval, ciphertext, ciphertext_log_10)  
approx.log_e(eval, ciphertext, ciphertext_log_e)

In [38]:
message_out = heaan.Message(log_slots)
message_out_log_2 = heaan.Message(log_slots)
message_out_log_10 = heaan.Message(log_slots)
message_out_log_e = heaan.Message(log_slots)
message_out_log_ = heaan.Message(log_slots)

dec.decrypt(ciphertext, sk, message_out)
dec.decrypt(ciphertext_log_, sk, message_out_log_)
dec.decrypt(ciphertext_log_2, sk, message_out_log_2)
dec.decrypt(ciphertext_log_10, sk, message_out_log_10)
dec.decrypt(ciphertext_log_e, sk, message_out_log_e)

print('message :', message_out)
print()
print('log :', message_out_log_)
print()
print('log_2 :', message_out_log_2)
print()
print('log_10 :', message_out_log_10)
print()
print('log_e : ', message_out_log_e)

message : [ (0.000000+0.000000j), (1.000000+0.000000j), (2.000000+0.000000j), (3.000000+0.000000j), (4.000000+0.000000j), (5.000000+0.000000j), (6.000000+0.000000j), (7.000000+0.000000j) ]

log : [ (4492393500690405.500000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (1.584962+0.000000j), (2.000000+0.000000j), (2.321928+0.000000j), (2.584962+0.000000j), (2.807355+0.000000j) ]

log_2 : [ (4492393500690405.500000+0.000000j), (0.000000+0.000000j), (1.000000+0.000000j), (1.584962+0.000000j), (2.000000+0.000000j), (2.321928+0.000000j), (2.584962+0.000000j), (2.807355+0.000000j) ]

log_10 : [ (-338088477213185.375000+0.000000j), (-0.000000+0.000000j), (0.301030+0.000000j), (0.477121+0.000000j), (0.602060+0.000000j), (0.698970+0.000000j), (0.778151+0.000000j), (0.845098+0.000000j) ]

log_e :  [ (3113889888969262.000000+0.000000j), (0.000000+0.000000j), (0.693147+0.000000j), (1.098612+0.000000j), (1.386294+0.000000j), (1.609438+0.000000j), (1.791759+0.000000j), (1.945910+0.000000j) 

### relu
- relu(arg0: piheaan.HomEvaluator, 
        arg1: piheaan.Ciphertext, 
        arg2: piheaan.Ciphertext)

In [39]:
data = np.random.uniform(2**(-18), 1, size=num_slots)
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]
print(message)

ciphertext = heaan.Ciphertext(context)
enc.encrypt(message, pk, ciphertext)

[ (0.456869+0.000000j), (0.660708+0.000000j), (0.018736+0.000000j), (0.059109+0.000000j), (0.752345+0.000000j), (0.865611+0.000000j), (0.121043+0.000000j), (0.810350+0.000000j) ]


In [40]:
def ReLU(x):
    return np.maximum(0, x)

In [41]:
plaintext_relu = []
for x in message:
    plaintext_relu.append(ReLU(x.real))

print(plaintext_relu)

[0.45686876362846274, 0.660708357280833, 0.0187358213923265, 0.05910887109426354, 0.7523451862334931, 0.8656108338326649, 0.12104286736335992, 0.8103496075073514]


In [42]:
result_relu = heaan.Ciphertext(context)

approx.relu(eval, ciphertext, result_relu)

result_relu_message = heaan.Message(log_slots)
dec.decrypt(result_relu, sk, result_relu_message)
print("relu : ", result_relu_message)

relu :  [ (0.474098+0.000000j), (0.649256+0.000000j), (0.027381+0.000000j), (0.053492+0.000000j), (0.735590+0.000000j), (0.871335+0.000000j), (0.104685+0.000000j), (0.801734+0.000000j) ]


### sigmoid
- sigmoid(arg0: piheaan.HomEvaluator, 
            arg1: piheaan.Ciphertext, 
            arg2: piheaan.Ciphertext, 
            arg3: float)

In [43]:
def sigmoid(x):
    sig = 1 / (1 + math.exp(-x))
    return sig

In [44]:
plaintext_sigmoid = []
for x in message_3:
    plaintext_sigmoid.append(sigmoid(x.real))

print(plaintext_sigmoid)

[0.5785041575568142, 0.6276315922436166, 0.5543482467420584, 0.7051447763933393, 0.541898096449181, 0.6278317305436136, 0.6078852034453884, 0.6997445892354243]


In [45]:
ciphertext_3 = heaan.Ciphertext(context)
result_sigmoid = heaan.Ciphertext(context)

enc.encrypt(message_3, pk, ciphertext_3)
enc.encrypt(message_4, pk, ciphertext_4)

approx.sigmoid(eval, ciphertext_3, result_sigmoid, 8.0)

result_sigmoid_message = heaan.Message(log_slots)
dec.decrypt(result_sigmoid, sk, result_sigmoid_message)
print("sigmoid : ", result_sigmoid_message)

sigmoid :  [ (0.578502+0.000000j), (0.627630+0.000000j), (0.554347+0.000000j), (0.705146+0.000000j), (0.541897+0.000000j), (0.627830+0.000000j), (0.607883+0.000000j), (0.699746+0.000000j) ]


### sign
- input range 2^-18 < |x| < 1
- sign(arg0: piheaan.HomEvaluator, 
       arg1: piheaan.Ciphertext, 
       arg2: piheaan.Ciphertext, 
       arg3: numiter_g : int, 
       arg4: numiter_f : int)

- default : 3rd paramter = 8 and 4th paramter = 3

In [46]:
data = np.random.uniform(-1, 1, size=num_slots)
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]
print(message)

ciphertext = heaan.Ciphertext(context)
enc.encrypt(message, pk, ciphertext)

[ (-0.623574+0.000000j), (0.208610+0.000000j), (-0.843150+0.000000j), (-0.575945+0.000000j), (-0.148376+0.000000j), (0.962112+0.000000j), (0.171445+0.000000j), (-0.382732+0.000000j) ]


In [47]:
result_sign = heaan.Ciphertext(context)

approx.sign(eval, ciphertext, result_sign)

result_sign_message = heaan.Message(log_slots)
dec.decrypt(result_sign, sk, result_sign_message)
print("sign : ", result_sign_message)

sign :  [ (-1.000000+0.000000j), (1.000000+0.000000j), (-1.000000+0.000000j), (-1.000000+0.000000j), (-1.000000+0.000000j), (1.000000+0.000000j), (1.000000+0.000000j), (-1.000000+0.000000j) ]


### min_max
- compare two ciphertexts and return two ciphertexts with minimum value or maximum value
- imnput range : 2^-18 < |x-y| < 1
- min_max(eval: piheaan.HomEvaluator, 
            op1: piheaan.Ciphertext, 
            op2: piheaan.Ciphertext, 
            min: piheaan.Ciphertext, 
            max: piheaan.Ciphertext, 
            numiter_g: int = 8, 
            numiter_f: int = 3) 

- default : 5 ~ 6th paramter

In [48]:
data3 = np.random.uniform(size=num_slots)
data4 = np.random.uniform(size=num_slots)
message_3 = heaan.Message(log_slots)
message_4 = heaan.Message(log_slots)
for i in range(num_slots):
    message_3[i]=data3[i]
    message_4[i]=data4[i]

print("message_3 : ", message_3)
print()
print("message_4 : ", message_4)

message_3 :  [ (0.369825+0.000000j), (0.943454+0.000000j), (0.048667+0.000000j), (0.795119+0.000000j), (0.080350+0.000000j), (0.268601+0.000000j), (0.798674+0.000000j), (0.747631+0.000000j) ]

message_4 :  [ (0.473638+0.000000j), (0.870304+0.000000j), (0.107515+0.000000j), (0.311041+0.000000j), (0.524878+0.000000j), (0.832252+0.000000j), (0.383240+0.000000j), (0.652498+0.000000j) ]


In [49]:
ciphertext_3 = heaan.Ciphertext(context)
ciphertext_4 = heaan.Ciphertext(context)

enc.encrypt(message_3, pk, ciphertext_3)
enc.encrypt(message_4, pk, ciphertext_4)

In [50]:
ciphertext_out_min = heaan.Ciphertext(context)
ciphertext_out_max = heaan.Ciphertext(context)

approx.min_max(eval, ciphertext_3, ciphertext_4, ciphertext_out_min, ciphertext_out_max)

In [51]:
message_out_min = heaan.Message(log_slots)
message_out_max = heaan.Message(log_slots)
dec.decrypt(ciphertext_out_min, sk, message_out_min)
dec.decrypt(ciphertext_out_max, sk, message_out_max)
print("min : ", message_out_min)
print()
print("max : ", message_out_max)

min :  [ (0.369825+0.000000j), (0.870304+0.000000j), (0.048667+0.000000j), (0.311041+0.000000j), (0.080350+0.000000j), (0.268601+0.000000j), (0.383240+0.000000j), (0.652498+0.000000j) ]

max :  [ (0.473638+0.000000j), (0.943454+0.000000j), (0.107515+0.000000j), (0.795119+0.000000j), (0.524878+0.000000j), (0.832252+0.000000j), (0.798674+0.000000j), (0.747631+0.000000j) ]


### sqrt
- input range : 2^-18 ≤ x ≤ 2
- sqrt(arg0: piheaan.HomEvaluator, 
       arg1: piheaan.Ciphertext, 
       arg2: piheaan.Ciphertext,
       arg3 : init)
       
- The default value for 3rd parameter is 17.
- You can specify a value when you change the default settings.
- As the value of the variable changes, the value of the square root and the value of the ciphertext level change.
- Error when entering 0 or non-integer values



### sqrt_inverse
- x -> 1/sqrt(x)
- input range : 1 ≤ x ≤ 2^22 or 2^-10 ≤ x ≤ 1
- sqrt_inverse(eval: piheaan.HomEvaluator, 
                op: piheaan.Ciphertext, 
                res: piheaan.Ciphertext, 
                init: Optional[float] = None, 
                num_iter: Optional[int] = None, 
                greater_than_one: bool = True)

- default values are init = pow(2, -9), num_iter=20 if greater_than_one==true, 
- otherwise init = pow(2, -5), num_iter=14


In [52]:
log_slots = 3
num_slots = 2 ** log_slots

data = np.random.uniform(2**(-18), 2, num_slots)
# data = [i for i in range(num_slots)]
message = heaan.Message(log_slots)
for i in range(num_slots):
    message[i] = data[i]
print('message :', message)

message : [ (0.476514+0.000000j), (0.241998+0.000000j), (1.103534+0.000000j), (1.994289+0.000000j), (0.207289+0.000000j), (0.020362+0.000000j), (1.810466+0.000000j), (1.964830+0.000000j) ]


In [53]:
ciphertext = heaan.Ciphertext(context)
enc.encrypt(message, pk, ciphertext)

ciphertext_sqrt = heaan.Ciphertext(context)
approx.sqrt(eval, ciphertext, ciphertext_sqrt)

ciphertext_sqrt_inv = heaan.Ciphertext(context)
approx.sqrt_inverse(eval, ciphertext, ciphertext_sqrt_inv) # 1 ~ 2^18

message_out_sqrt = heaan.Message(log_slots)
message_out_sqrt_inv = heaan.Message(log_slots)

In [54]:
dec.decrypt(ciphertext_sqrt, sk, message_out_sqrt)
dec.decrypt(ciphertext_sqrt_inv, sk, message_out_sqrt_inv)

print('sqrt : ', message_out_sqrt)
print()
print('sqrt_inverse : ', message_out_sqrt_inv)

sqrt :  [ (0.690300+0.000000j), (0.491933+0.000000j), (1.050492+0.000000j), (1.412193+0.000000j), (0.455290+0.000000j), (0.142696+0.000000j), (1.345536+0.000000j), (1.401724+0.000000j) ]

sqrt_inverse :  [ (1.448645+0.000000j), (2.032559+0.000000j), (0.951935+0.000000j), (0.708118+0.000000j), (2.195650+0.000000j), (5.274408+0.000000j), (0.743198+0.000000j), (0.713407+0.000000j) ]
